In [1]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark Transformations").getOrCreate()
from pyspark.sql.functions import col,sum,avg,count,row_number
from pyspark.sql.window import Window

In [13]:
from google.colab import files
uploaded = files.upload()

Saving txn.csv to txn.csv


In [14]:
txn_df = spark.read.csv("txn.csv", header =True, inferSchema=True)
txn_df.show(5)

+-------------+--------------------+----------+----------------+-------------+-----------+
|   Account No| TRANSACTION DETAILS|VALUE DATE| WITHDRAWAL AMT | DEPOSIT AMT |BALANCE AMT|
+-------------+--------------------+----------+----------------+-------------+-----------+
|409000611074'|TRF FROM  Indiafo...| 29-Jun-17|            NULL|    1000000.0|  1000000.0|
|409000611074'|TRF FROM  Indiafo...|  5-Jul-17|            NULL|    1000000.0|  2000000.0|
|409000611074'|FDRL/INTERNAL FUN...| 18-Jul-17|            NULL|     500000.0|  2500000.0|
|409000611074'|TRF FRM  Indiafor...|  1-Aug-17|            NULL|    3000000.0|  5500000.0|
|409000611074'|FDRL/INTERNAL FUN...| 16-Aug-17|            NULL|     500000.0|  6000000.0|
+-------------+--------------------+----------+----------------+-------------+-----------+
only showing top 5 rows



In [5]:
loan_df = spark.read.csv("loan.csv", header =True, inferSchema=True)
loan_df.show(5)

+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450|      27675|            

TRANSFORMATIONS

FILTER

In [6]:
high_income = loan_df.filter(col("Income") > 60000)
high_income.show(5)

+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14032| 24|  MALE|       DATA ANALYST|        SINGLE|          4| 60111|      28999|            6|   AUTOMOBILE|     35,232|      5|      33,333|               1|                 2|
|    IB14042| 25|FEMALE|             DOCTOR|        SINGLE|          4| 60111|      27111|            5|   TRAVELLING| 12,90,929 |      4|      18,000|               1|                 0|
|    IB14082| 60|FEMALE|            TEACHER|       MARRIED| 

In [8]:
credit_df = spark.read.csv("credit card.csv",header=True,inferSchema=True)
credit_df.show(5)

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

GROUP BY


In [34]:
loan_df.groupBy("Loan Category").count().show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



JOINS

Left Join

In [17]:
from pyspark.sql.functions import regexp_extract, col

# Extract number from 'C001' → 1
loan_df_clean = loan_df.withColumn("CustomerId", regexp_extract("Customer_ID", r'\d+', 0).cast("int"))

# Load credit card data
credit_df = spark.read.csv("credit card.csv", header=True, inferSchema=True)

# Now join on numeric CustomerId
joined_df = loan_df_clean.join(credit_df, on="CustomerId", how="left")

# Show result
joined_df.select("Customer_ID", "Surname", "Loan Amount", "Balance").show(5)


+-----------+-------+-----------+-------+
|Customer_ID|Surname|Loan Amount|Balance|
+-----------+-------+-----------+-------+
|    IB14001|   NULL| 10,00,000 |   NULL|
|    IB14008|   NULL|     50,000|   NULL|
|    IB14012|   NULL|     75,000|   NULL|
|    IB14018|   NULL|  6,00,000 |   NULL|
|    IB14022|   NULL|  2,00,000 |   NULL|
+-----------+-------+-----------+-------+
only showing top 5 rows



Inner Join

In [32]:
loan_df_clean.join(credit_df, on="CustomerId", how="inner").select("Customer_ID", "Surname", "Balance").show()


+-----------+-------+-------+
|Customer_ID|Surname|Balance|
+-----------+-------+-------+
+-----------+-------+-------+



Left Join

In [30]:
loan_df_clean.join(credit_df, on="CustomerId", how="left").select("Customer_ID", "Surname", "Balance").show()


+-----------+-------+-------+
|Customer_ID|Surname|Balance|
+-----------+-------+-------+
|    IB14001|   NULL|   NULL|
|    IB14008|   NULL|   NULL|
|    IB14012|   NULL|   NULL|
|    IB14018|   NULL|   NULL|
|    IB14022|   NULL|   NULL|
|    IB14024|   NULL|   NULL|
|    IB14025|   NULL|   NULL|
|    IB14027|   NULL|   NULL|
|    IB14029|   NULL|   NULL|
|    IB14031|   NULL|   NULL|
|    IB14032|   NULL|   NULL|
|    IB14034|   NULL|   NULL|
|    IB14037|   NULL|   NULL|
|    IB14039|   NULL|   NULL|
|    IB14041|   NULL|   NULL|
|    IB14042|   NULL|   NULL|
|    IB14045|   NULL|   NULL|
|    IB14049|   NULL|   NULL|
|    IB14050|   NULL|   NULL|
|    IB14054|   NULL|   NULL|
+-----------+-------+-------+
only showing top 20 rows



Right Join

In [33]:
loan_df_clean.join(credit_df, on="CustomerId", how="right").select("Customer_ID", "Surname", "Balance").show()


+-----------+---------+---------+
|Customer_ID|  Surname|  Balance|
+-----------+---------+---------+
|       NULL| Hargrave|      0.0|
|       NULL|     Hill| 83807.86|
|       NULL|     Onio| 159660.8|
|       NULL|     Boni|      0.0|
|       NULL| Mitchell|125510.82|
|       NULL|      Chu|113755.78|
|       NULL| Bartlett|      0.0|
|       NULL|   Obinna|115046.74|
|       NULL|       He|142051.07|
|       NULL|       H?|134603.88|
|       NULL|   Bearce|102016.72|
|       NULL|  Andrews|      0.0|
|       NULL|      Kay|      0.0|
|       NULL|     Chin|      0.0|
|       NULL|    Scott|      0.0|
|       NULL|  Goforth|143129.41|
|       NULL|    Romeo|132602.88|
|       NULL|Henderson|      0.0|
|       NULL|  Muldrow|      0.0|
|       NULL|      Hao|      0.0|
+-----------+---------+---------+
only showing top 20 rows



AGGREGATION

In [19]:
loan_df = loan_df.withColumn("Loan Amount", col("Loan Amount").cast("int"))

loan_df.groupBy("Marital Status").agg(
    sum("Loan Amount").alias("TotalLoan"),
    avg("Loan Amount").alias("AvgLoan")
).show()

+--------------+---------+-------+
|Marital Status|TotalLoan|AvgLoan|
+--------------+---------+-------+
|        SINGLE|     NULL|   NULL|
|       MARRIED|     NULL|   NULL|
+--------------+---------+-------+



Max & Min Income per Occupation



In [27]:
loan_df.groupBy("Occupation").agg(
    {"Income": "max", "Income": "min"}
).show()


+--------------------+-----------+
|          Occupation|min(Income)|
+--------------------+-----------+
|      CIVIL ENGINEER|      46880|
|     FIRE DEPARTMENT|      40000|
|          ACCOUNTANT|      43350|
|        BANK MANAGER|      29565|
|      SYSTEM OFFICER|      56780|
|           NUTRITION|      55650|
|           DIETICIAN|      40060|
|               CLERK|      32714|
|   SOFTWARE ENGINEER|      30454|
|AGRICULTURAL ENGI...|      66610|
|   ASSISTANT MANAGER|      45612|
|             TEACHER|      30801|
| ASSISTANT PROFESSOR|      34999|
|     SYSTEM ENGINEER|      52364|
| CHARTERED APPRAISER|      58661|
|                NAVY|      39213|
|              POLICE|      33312|
|            BUSINESS|      34444|
|              FARMER|      51162|
|              DRIVER|      30000|
+--------------------+-----------+
only showing top 20 rows



Standard Deviation of Expenditure by Occupation



In [28]:
from pyspark.sql.functions import stddev

loan_df.groupBy("Occupation").agg(stddev("Expenditure").alias("stddev_exp")).show()


+--------------------+------------------+
|          Occupation|        stddev_exp|
+--------------------+------------------+
|      CIVIL ENGINEER| 5841.266041421728|
|     FIRE DEPARTMENT| 6316.349616093719|
|          ACCOUNTANT| 5374.031066598136|
|        BANK MANAGER|11217.422514451566|
|      SYSTEM OFFICER| 4707.928445717925|
|           NUTRITION|              NULL|
|           DIETICIAN|11835.213718781371|
|               CLERK| 6326.715487153237|
|   SOFTWARE ENGINEER|  7655.72613366057|
|AGRICULTURAL ENGI...|12875.871787405862|
|   ASSISTANT MANAGER|  7213.34187923092|
|             TEACHER| 8613.057535672191|
| ASSISTANT PROFESSOR| 9242.661288527479|
|     SYSTEM ENGINEER|               0.0|
| CHARTERED APPRAISER| 9296.023820576585|
|                NAVY| 9598.032914569874|
|              POLICE|11188.968716605497|
|            BUSINESS| 9747.383497636687|
|              FARMER|12349.656280159996|
|              DRIVER| 8545.589693279355|
+--------------------+------------

Collect List of Loan Categories by Marital Status

In [29]:
from pyspark.sql.functions import collect_list

loan_df.groupBy("Marital Status").agg(collect_list("Loan Category").alias("loan_types")).show(truncate=False)


+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

WINDOW FUNCTION

In [20]:
windowSpec = Window.partitionBy("Occupation").orderBy(col("Loan Amount").desc())

ranked_df = loan_df.withColumn("rank", row_number().over(windowSpec))
ranked_df.filter(col("rank") == 1).select("Customer_ID", "Occupation", "Loan Amount").show()


+-----------+--------------------+-----------+
|Customer_ID|          Occupation|Loan Amount|
+-----------+--------------------+-----------+
|    IB14039|     ACCOUNT MANAGER|       NULL|
|    IB14604|          ACCOUNTANT|       NULL|
|    IB14611|AGRICULTURAL ENGI...|       NULL|
|    IB14057|     AIRPORT OFFICER|       NULL|
|    IB14620|                ARMY|       NULL|
|    IB14113|   ASSISTANT MANAGER|       NULL|
|    IB14041| ASSISTANT PROFESSOR|       NULL|
|    IB14001|        BANK MANAGER|       NULL|
|    IB14565|            BUSINESS|       NULL|
|    IB14566| CHARTERED APPRAISER|       NULL|
|    IB14050|      CIVIL ENGINEER|       NULL|
|    IB14060|               CLERK|       NULL|
|    IB14111|   CORPORATE OFFICER|       NULL|
|    IB14032|        DATA ANALYST|       NULL|
|    IB14623|       DATA ENGINEER|       NULL|
|    IB14012|             DENTIST|       NULL|
|    IB14597|           DIETICIAN|       NULL|
|    IB14042|              DOCTOR|       NULL|
|    IB14109|

In [26]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank

# Define a window partitioned by Gender and ordered by Income
window_spec = Window.partitionBy("Gender").orderBy(col("Income").desc())

# 1. Row number within each gender
loan_df.withColumn("row_num", row_number().over(window_spec)).show()

# 2. Rank of income within each gender
loan_df.withColumn("income_rank", rank().over(window_spec)).show()

# 3. Dense rank (no gaps in ranking)
loan_df.withColumn("income_dense_rank", dense_rank().over(window_spec)).show()


+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+-------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|     Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|row_num|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+------------------+-----------+-------+------------+----------------+------------------+-------+
|    IB14107| 44|FEMALE|    ACCOUNT MANAGER|       MARRIED|          4|800000|      15632|            8|        AUTOMOBILE|       NULL|      5|      20,145|               3|                 4|      1|
|    IB14163| 44|FEMALE|    ACCOUNT MANAGER|       MARRIED|          4|800000|      15632|            8|COMPUTER SOFTWARES|       NULL|      5|      20,145|               3|                 4|    

ACTION IN PYSPARK

SHOW()

In [21]:
loan_df.show(3)

+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING|       NULL|      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|       NULL|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450|      27675|            

COUNT()

In [22]:
loan_df.count()

500

COLLECT()

In [23]:
loan_df.collect()

[Row(Customer_ID='IB14001', Age=30, Gender='MALE', Occupation='BANK MANAGER', Marital Status='SINGLE', Family Size=4, Income=50000, Expenditure=22199, Use Frequency=6, Loan Category='HOUSING', Loan Amount=None, Overdue=5,  Debt Record='42,898',  Returned Cheque=6,  Dishonour of Bill=9),
 Row(Customer_ID='IB14008', Age=44, Gender='MALE', Occupation='PROFESSOR', Marital Status='MARRIED', Family Size=6, Income=51000, Expenditure=19999, Use Frequency=4, Loan Category='SHOPPING', Loan Amount=None, Overdue=3,  Debt Record='33,999',  Returned Cheque=1,  Dishonour of Bill=5),
 Row(Customer_ID='IB14012', Age=30, Gender='FEMALE', Occupation='DENTIST', Marital Status='SINGLE', Family Size=3, Income=58450, Expenditure=27675, Use Frequency=5, Loan Category='TRAVELLING', Loan Amount=None, Overdue=6,  Debt Record='20,876',  Returned Cheque=3,  Dishonour of Bill=1),
 Row(Customer_ID='IB14018', Age=29, Gender='MALE', Occupation='TEACHER', Marital Status='MARRIED', Family Size=5, Income=45767, Expenditu

FIRST()

In [24]:
loan_df.first()

Row(Customer_ID='IB14001', Age=30, Gender='MALE', Occupation='BANK MANAGER', Marital Status='SINGLE', Family Size=4, Income=50000, Expenditure=22199, Use Frequency=6, Loan Category='HOUSING', Loan Amount=None, Overdue=5,  Debt Record='42,898',  Returned Cheque=6,  Dishonour of Bill=9)

TAKE()

In [25]:
loan_df.take(4)

[Row(Customer_ID='IB14001', Age=30, Gender='MALE', Occupation='BANK MANAGER', Marital Status='SINGLE', Family Size=4, Income=50000, Expenditure=22199, Use Frequency=6, Loan Category='HOUSING', Loan Amount=None, Overdue=5,  Debt Record='42,898',  Returned Cheque=6,  Dishonour of Bill=9),
 Row(Customer_ID='IB14008', Age=44, Gender='MALE', Occupation='PROFESSOR', Marital Status='MARRIED', Family Size=6, Income=51000, Expenditure=19999, Use Frequency=4, Loan Category='SHOPPING', Loan Amount=None, Overdue=3,  Debt Record='33,999',  Returned Cheque=1,  Dishonour of Bill=5),
 Row(Customer_ID='IB14012', Age=30, Gender='FEMALE', Occupation='DENTIST', Marital Status='SINGLE', Family Size=3, Income=58450, Expenditure=27675, Use Frequency=5, Loan Category='TRAVELLING', Loan Amount=None, Overdue=6,  Debt Record='20,876',  Returned Cheque=3,  Dishonour of Bill=1),
 Row(Customer_ID='IB14018', Age=29, Gender='MALE', Occupation='TEACHER', Marital Status='MARRIED', Family Size=5, Income=45767, Expenditu